# Adaptive Tracking Simulation Example

## Setup

In [10]:
from datetime import datetime, timedelta
start_time = datetime.now()

## Radar system

In [11]:
from mpar_sim.radar import PhasedArrayRadar
from mpar_sim.beam.beam import RectangularBeam, GaussianBeam
from mpar_sim.look import RadarLook
from mpar_sim.resource_management import PAPResourceManager
from mpar_sim.schedulers import PriorityScheduler
import numpy as np

radar = PhasedArrayRadar(
    position=np.array([[0], [0], [0]]),
    position_mapping=(0, 2, 4),
    rotation_offset=np.array([[0], [0], [0]]),
    # Array parameters
    n_elements_x=16,
    n_elements_y=16,
    element_spacing=0.5,  # Wavelengths
    element_tx_power=10,
    # System parameters
    center_frequency=3e9,
    system_temperature=290,
    noise_figure=4,
    # Scan settings
    beam_shape=GaussianBeam,
    az_fov=[-90, 90],
    el_fov=[-90, 90],
    # Detection settings
    false_alarm_rate=1e-6,
)
radar.timestamp = start_time

# resource_manager = PAPResourceManager(radar,
#                                       max_duty_cycle=0.1,
#                                       max_bandwidth=100e6)
# scheduler = 

Raster scan agent

In [12]:
from mpar_sim.agents.raster_scan import RasterScanAgent
from mpar_sim.agents.adaptive_track import AdaptiveTrackAgent
import numpy as np

search_agent = RasterScanAgent(
    azimuth_scan_limits=np.array([-30, 30]),
    elevation_scan_limits=np.array([-5, 5]),
    azimuth_beam_spacing=0.85,
    elevation_beam_spacing=0.85,
    azimuth_beamwidth=10,
    elevation_beamwidth=10,
    bandwidth=100e6,
    pulsewidth=1e-6,
    prf=5e3,
    n_pulses=100,
)

track_agent = AdaptiveTrackAgent(
    # Adaptive track parameters 
    track_sharpness=0.05,
    min_revisit_rate=0.5,
    max_revisit_rate=5,
    confirm_rate=20,
    # Task parameters
    azimuth_beamwidth=5,
    elevation_beamwidth=5,
    bandwidth=100e6,
    pulsewidth=1e-6,
    prf=5e3,
    n_pulses=100
)

## Tracker Components

Create tracker

In [13]:
from stonesoup.measures import Mahalanobis, Euclidean
from stonesoup.hypothesiser.distance import DistanceHypothesiser
from stonesoup.updater.kalman import ExtendedKalmanUpdater
from stonesoup.predictor.kalman import KalmanPredictor
from stonesoup.models.transition.linear import CombinedLinearGaussianTransitionModel, ConstantVelocity
from stonesoup.gater.distance import DistanceGater

# KF prediction model. Assuming it's matched to the true target model for now
transition_model = CombinedLinearGaussianTransitionModel([
    ConstantVelocity(10),
    ConstantVelocity(10),
    ConstantVelocity(0.0),
])
predictor = KalmanPredictor(transition_model)

updater = ExtendedKalmanUpdater(measurement_model=None)

hypothesizer = DistanceHypothesiser(
    predictor, updater, measure=Mahalanobis(), missed_distance=100)
gater = DistanceGater(hypothesizer, measure=Mahalanobis(), gate_threshold=15)


Create the data associator

In [14]:
from stonesoup.dataassociator.neighbour import GNNWith2DAssignment
data_associator = GNNWith2DAssignment(gater)

Create the deleter

In [15]:
from stonesoup.deleter.time import UpdateTimeStepsDeleter, UpdateTimeDeleter
# deleter = UpdateTimeDeleter(timedelta(0.5))
deleter = UpdateTimeStepsDeleter(int(1.5*search_agent.n_positions))

Create the initiator

In [16]:
# from mpar_sim.
from stonesoup.types.state import GaussianState
from stonesoup.initiator.simple import MultiMeasurementInitiator, SimpleMeasurementInitiator
import numpy as np
from mpar_sim.initiator.initators import MofNInitiator


initiator = MofNInitiator(
    prior_state=GaussianState([0, 0, 0, 0, 0, 0], np.diag([0, 0, 0, 0, 0, 0])),
    measurement_model=None,
    deleter=deleter,
    data_associator=data_associator,
    updater=updater,
    confirmation_threshold=[3,2*search_agent.n_positions],
)

## Run the simulation

In [17]:
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.array import CovarianceMatrix
from stonesoup.types.state import StateVector
import random
# Set the simulation seed
seed = np.random.randint(0, 2**32-1)
np.random.seed(seed)
random.seed(seed)

# Simulation-level parameters
n_steps = 100
include_noise = False

# Target generation parameters
n_targets_max = 100
initial_state_mean = StateVector([0, 0, 0, 0, 0, 0])
initial_state_covariance = CovarianceMatrix(np.diag([100, 5, 100, 5, 0, 0]))
initial_state = GaussianState(initial_state_mean, initial_state_covariance)
death_probability = 0.01
birth_probability = 0.8
target_rcs = 1


truths = []
all_truths = []

confirmed_tracks = set()
tentative_tracks = set()
all_measurements = []
all_tracks = set()

time = start_time
for istep in range(n_steps):
  measurements = set()

  ########################################
  # Target birth/death
  ########################################
  # Delete targets according to the death process
  truths = [truth for truth in truths if np.random.rand() > death_probability]
  # Also randomly delete targets if we have exceeded the maximum target count
  if len(truths) > n_targets_max:
    indices = np.random.choice(
        len(truths), len(truths) - n_targets_max, replace=False)
    truths = truths[indices]

  # Targets, be reborn!
  for _ in range(np.random.poisson(birth_probability)):
    # Sample an initial state from the mean and covariance defined above
    state_vector = initial_state.state_vector + \
        initial_state.covar @ np.random.randn(initial_state.ndim, 1)
    state = GroundTruthState(
        state_vector=state_vector,
        timestamp=time,
    )

    # Give the target an RCS
    # TODO: Create a GroundTruthTarget class with an RCS attribute
    state.rcs = target_rcs
    # Add to the list of truths
    truth = GroundTruthPath([state])
    truths.append(truth)
    all_truths.append(truth)

  ########################################
  # Allocate resources and simulate
  ########################################
  # Look for search task
  search_look = search_agent.act(current_time=time)

  # Feed tracks to the adaptive track agent for scheduling
  track_looks = track_agent.request_looks(
      confirmed_tracks, tentative_tracks, time, predictor)

  # Choose an action and allocate resources
  look = search_look
  look.tx_power = 100e3
  radar.load_look(look)

  # Simulate detections
  measurements |= radar.measure(truths, noise=include_noise)

  ########################################
  # Update tracks
  ########################################
  # TODO: For anything but TWS, this should not happen on volume search dwells (just confirm/updates). I need to think about it a little more, but it's possible that only the else statement should be excluded from volume dwells so we can use measurements if we get them.
  # Calculate all hypothesis pairs and associate the elements in the best subset to the tracks.
  hypotheses = data_associator.associate(confirmed_tracks,
                                         measurements,
                                         timestamp=time)
  associated_measurements = set()
  # Update confirmed tracks with new measurements
  for track in confirmed_tracks:
    hypothesis = hypotheses[track]
    if hypothesis.measurement:
      post = updater.update(hypothesis)
      track.append(post)
      associated_measurements.add(hypothesis.measurement)
    else:
      # When data associator says no detections are good enough, we'll keep the prediction
      track.append(hypothesis.prediction)

  # Carry out deletion and initiation on the list of confirmed tracks
  confirmed_tracks -= deleter.delete_tracks(confirmed_tracks)
  confirmed_tracks |= initiator.initiate(
      detections=measurements - associated_measurements,
      timestamp=time)

  tentative_tracks = initiator.holding_tracks

  ########################################
  # Advance simulation
  ########################################
  # Advance simulation time
  # TODO: This gets more complicated when multiple tasks with different dwell times are scheduled
  dt = timedelta(seconds=look.dwell_time)
  time += dt

  # Advance target positions
  for truth in truths:
    truth.append(GroundTruthState(
        transition_model.function(truth[-1],
                                  noise=include_noise,
                                  time_interval=dt),
        timestamp=time))
    truth[-1].rcs = target_rcs

  # Save measurements and tracks for plotting
  all_measurements.append(measurements)
  all_tracks |= confirmed_tracks


## Plot simulation results

In [18]:
from stonesoup.plotter import Plotterly

plotter = Plotterly()
plotter.plot_sensors(radar, "Radar")
plotter.plot_ground_truths(all_truths, [0, 2])
plotter.plot_measurements(all_measurements, [0, 2])
plotter.plot_tracks(all_tracks, [0,2])

plotter.fig